In [34]:
import pandas as pd
import numpy as np
import MICE
import iMICE

In [35]:
df = pd.read_csv('titanic.csv')
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)
df = df.reset_index(drop = True)

df_1=df.copy()

actual = df_1['Age'].copy()
update = df_1.sample(int(0.1 * len(df))).index  #0.1 = 10%
df_1.at[update, 'Age'] = np.nan # changable

myFilter_train=df_1.Age.notnull()

y_train=actual[myFilter_train]
y_test=actual[update]

x_pclass=df['Pclass']
x_pclass_1=x_pclass[myFilter_train]
x_fare=df['Fare']
x_fare_1=x_fare[myFilter_train]
X_train=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})

x_pclass=df['Pclass']
x_pclass_1=x_pclass[update]
x_fare=df['Fare']
x_fare_1=x_fare[update]
X_test=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})

In [36]:
df_2 = df_1[['Age', 'Pclass', 'Fare']].copy()

In [37]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [38]:
df_3 = pd.DataFrame(mice_result, columns = ['Age', 'Pclass', 'Fare'])
predict = df_3['Age']

In [39]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(actual[update], predict[update]))  
print('Mean Squared Error:', metrics.mean_squared_error(actual[update], predict[update]))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(actual[update], predict[update]))) 

Mean Absolute Error: 17.12676056338028
Mean Squared Error: 456.80281690140845
Root Mean Squared Error: 21.372945910692994


# iMICE

In [40]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [41]:
df_3i = pd.DataFrame(imice_result, columns = ['Age', 'Pclass', 'Fare'])
predicti = df_3i['Age']

In [42]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(actual[update], predicti[update]))  
print('Mean Squared Error:', metrics.mean_squared_error(actual[update], predicti[update]))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(actual[update], predicti[update]))) 

Mean Absolute Error: 11.767605633802816
Mean Squared Error: 215.79225352112675
Root Mean Squared Error: 14.689869077739486


# Random Forest

In [43]:
X_test=X_test.iloc[:,:].values
X_train=X_train.iloc[:,:].values
y_train=np.array(y_train)
y_test=np.array(y_test)

In [44]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
regressor = RandomForestRegressor(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test) 

In [45]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 12.069834162221936
Mean Squared Error: 223.0392635781795
Root Mean Squared Error: 14.93449910704003


# SVR

In [46]:
from sklearn.svm import SVR
svr = SVR(gamma=0.001, C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred=svr.predict(X_test)

In [47]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 10.666376657806026
Mean Squared Error: 188.09713421227778
Root Mean Squared Error: 13.714850863654252
